In [ ]:
!pip install tensorflow==2.13.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 21.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: gast
    Found existing installation: gast 0.6.0
    Uninstalling gast-0.6.0:
      

In [ ]:
!pip install gradio

  Using cached gradio-4.40.0-py3-none-any.whl.metadata (15 kB)
  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached fastapi-0.112.0-py3-none-any.whl.metadata (27 kB)
  Using cached ffmpy-0.4.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached gradio_client-1.2.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached orjson-3.10.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (50 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.9-py3-none-any.whl.metadata (2.5 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.6/318.6 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
from PIL import Image

In [ ]:
beta = 1.0

# Loss for reveal network
def rev_loss(s_true, s_pred):
    # Loss for reveal network is: beta * |S-S'|
    return beta * K.sum(K.square(s_true - s_pred))

# Loss for the full model, used for preparation and hidding networks
def full_loss(y_true, y_pred):
    # Loss for the full model is: |C-C'| + beta * |S-S'|
    s_true, c_true = y_true[...,0:3], y_true[...,3:6]
    s_pred, c_pred = y_pred[...,0:3], y_pred[...,3:6]

    s_loss = rev_loss(s_true, s_pred)
    c_loss = K.sum(K.square(c_true - c_pred))
    return s_loss + c_loss

In [ ]:
import tensorflow as tf
print(tf.__version__)


2.13.0


In [ ]:
!pip uninstall tensorflow -y


Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Successfully uninstalled tensorflow-2.17.0


In [ ]:
with tf.device('/GPU:0'):
    model = tf.keras.models.load_model("model.h5", custom_objects={'full_loss': full_loss})

In [ ]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 124, 124, 3)]        0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 124, 124, 3)]        0         []                            
                                                                                                  
 Encoder (Functional)        (None, 124, 124, 3)          293273    ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
 Decoder (Functional)        (None, 124, 124, 3)          195388    ['Encoder[0][0]']         

In [ ]:
!pip uninstall pydantic typing_extensions -y


Found existing installation: pydantic 2.8.2
Uninstalling pydantic-2.8.2:
  Successfully uninstalled pydantic-2.8.2
Found existing installation: typing_extensions 4.12.2
Uninstalling typing_extensions-4.12.2:
  Successfully uninstalled typing_extensions-4.12.2


In [ ]:

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def preprocess_image(img):
    if isinstance(img, np.ndarray):
        img = Image.fromarray(img)
    img = img.resize((124, 124), Image.ANTIALIAS)
    img = np.array(img)
    img = img / 255.0
    return img

def steganography_image(imageO, imageF):
    # Preprocess images
    img_S = preprocess_image(imageO)
    img_C = preprocess_image(imageF)

    # Add batch dimension
    img_S = np.expand_dims(img_S, axis=0)
    img_C = np.expand_dims(img_C, axis=0)

    # Predict with pre/loaded model
    decoded = model.predict([img_S, img_C])
    decoded_S, decoded_C = decoded[..., 0:3], decoded[..., 3:6]

    # Post-process outputs
    decoded_S = np.squeeze(decoded_S, axis=0)  # Remove batch dimension
    decoded_C = np.squeeze(decoded_C, axis=0)  # Remove batch dimension
    decoded_S = (decoded_S * 255).astype(np.uint8)
    decoded_C = (decoded_C * 255).astype(np.uint8)

    # Calculate absolute differences
    diff_S = np.abs(decoded_S - (img_S.squeeze() * 255)).astype(np.uint8)
    diff_C = np.abs(decoded_C - (img_C.squeeze() * 255)).astype(np.uint8)

    # Create a plot of differences
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(diff_S)
    ax[0].set_title('Difference in Secret Image')
    ax[0].axis('off')
    ax[1].imshow(diff_C)
    ax[1].set_title('Difference in Cover Image')
    ax[1].axis('off')
    plt.tight_layout()

    # Return images and plot
    return decoded_S, decoded_C, fig


In [ ]:
import cv2
import os
import shutil

#Function to clear a folder
def clear_folder(path):
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)

#Function to extract every frame of a video and save them in a folder
def extractImages(pathIn, pathOut):
    clear_folder(pathOut)
    if not os.path.exists(pathOut):
        os.makedirs(pathOut)

    vidcap = cv2.VideoCapture(pathIn)
    success, image = vidcap.read()
    count = 0

    while success:
        frame_path = os.path.join(pathOut, f"frame{count}.jpg")
        success, image = vidcap.read()

        if success:
            resized_image = cv2.resize(image, (124, 124), interpolation=cv2.INTER_AREA)
            cv2.imwrite(frame_path, resized_image)
            print(f'Saved frame {count} to {frame_path}')
        else:
            print(f'Failed to read frame at count {count}')

        count += 1


In [ ]:
#Function to create a new video based on a folder of frames
def rebuildVideo(framesPath, outputPath, fps=30):
    frame_files = sorted([f for f in os.listdir(framesPath) if f.endswith('.jpg')],
                         key=lambda x: int(x[5:-4]))

    first_frame_path = os.path.join(framesPath, frame_files[0])
    frame = cv2.imread(first_frame_path)
    height, width, layers = frame.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(outputPath, fourcc, fps, (width, height))

    for file in frame_files:
        frame_path = os.path.join(framesPath, file)
        frame = cv2.imread(frame_path)
        out.write(frame)
    out.release()



In [ ]:
output_frames_path = "/content/drive/MyDrive/Machine Learning Techniques/Proyecto/Frames3"
output_frames_path2 = "/content/drive/MyDrive/Machine Learning Techniques/Proyecto/Frames4"
rebuildVideo(output_frames_path, "/content/drive/MyDrive/Machine Learning Techniques/Proyecto/Examples/coverVideo.mp4")
rebuildVideo(output_frames_path2, "/content/drive/MyDrive/Machine Learning Techniques/Proyecto/Examples/secretVideo.mp4")

In [ ]:
from skimage.metrics import structural_similarity as ssim
def calculate_ssim(img1, img2):
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    score, _ = ssim(img1_gray, img2_gray, full=True)
    return score

def plot_metrics(metrics):
    fig, ax = plt.subplots()
    ax.plot(metrics, label="SSIM")
    ax.set_xlabel("Frame")
    ax.set_ylabel("SSIM")
    ax.set_title("SSIM over Frames")
    ax.legend()
    ax.grid(True)
    return fig

In [ ]:
def process_frame(imageO, imageF):
    img_S = preprocess_image(imageO)
    img_C = preprocess_image(imageF)
    img_S = np.expand_dims(img_S, axis=0)
    img_C = np.expand_dims(img_C, axis=0)
    decoded = model.predict([img_S, img_C])
    decoded_S, decoded_C = decoded[..., 0:3], decoded[..., 3:6]
    decoded_S = np.squeeze(decoded_S, axis=0)
    decoded_C = np.squeeze(decoded_C, axis=0)
    decoded_S = (decoded_S * 255).astype(np.uint8)
    decoded_C = (decoded_C * 255).astype(np.uint8)
    return decoded_S, decoded_C

def steganography_video(video_path1, video_path2):
    input_frames_path = "/content/drive/MyDrive/Machine Learning Techniques/Proyecto/Frames1"
    input_frames_path2 = "/content/drive/MyDrive/Machine Learning Techniques/Proyecto/Frames2"
    output_frames_path = "/content/drive/MyDrive/Machine Learning Techniques/Proyecto/Frames3"
    output_frames_path2 = "/content/drive/MyDrive/Machine Learning Techniques/Proyecto/Frames4"
    output_video_path = "output_video.mp4"
    output_video_path2 = "output_video2.mp4"
    extractImages(video_path1, input_frames_path)
    extractImages(video_path2, input_frames_path2)

    input_frame_files = sorted([f for f in os.listdir(input_frames_path) if f.endswith('.jpg')],
                               key=lambda x: int(x[5:-4]))

    clear_folder(output_frames_path)
    clear_folder(output_frames_path2)
    i = 0
    ssim_scores = []
    ssim_scores2 = []

    for file in input_frame_files:
        frame_path = os.path.join(input_frames_path, file)
        frame_path2 = os.path.join(input_frames_path2, f"frame{i}.jpg")
        frame = cv2.imread(frame_path)
        try:
          frame2 = cv2.imread(frame_path2)
        except:
          print("Second video is too short, will be cut up to the length of the first one")
          break
        if frame2 is None:
          break
        decoded_S, decoded_C = process_frame(frame, frame2)
        decoded_S_path = os.path.join(output_frames_path, file)
        cv2.imwrite(decoded_S_path, decoded_S)
        decoded_C_path = os.path.join(output_frames_path2, file)
        cv2.imwrite(decoded_C_path, decoded_C)
        print(frame.shape)
        print(decoded_S.shape)
        print(frame2.shape)
        print(decoded_C.shape)
        ssim_scores.append(calculate_ssim(frame, decoded_S))
        ssim_scores2.append(calculate_ssim(frame2, decoded_C))
        i+=1

    rebuildVideo(output_frames_path, output_video_path, fps=20)
    rebuildVideo(output_frames_path2, output_video_path2, fps=20)

    return output_video_path, output_video_path2, ssim_scores, ssim_scores2


In [ ]:
import gradio as gr
example_secret_image = "/content/drive/MyDrive/Machine Learning Techniques/Proyecto/Examples/secret.jpg"
example_cover_image = "/content/drive/MyDrive/Machine Learning Techniques/Proyecto/Examples/cover.jpg"
example_cover_video = "/content/drive/MyDrive/Machine Learning Techniques/Proyecto/Examples/cover.mp4"
example_secret_video = "/content/drive/MyDrive/Machine Learning Techniques/Proyecto/Examples/secret.mp4"

with gr.Blocks() as demo:
    with gr.Tab("Image Processing"):
        image_input1 = gr.Image(label="Cover Image", value=example_cover_image)
        image_input2 = gr.Image(label="Secret Image", value=example_secret_image)
        image_output1 = gr.Image(label="Decoded Cover Image")
        image_output2 = gr.Image(label="Decoded Secret Image")
        plot = gr.Plot(label = "Noise behind each image")
        btn_image = gr.Button("Process Images")

        btn_image.click(
            fn=steganography_image,
            inputs=[image_input1, image_input2],
            outputs=[image_output1, image_output2, plot]
        )

    with gr.Tab("Video Processing"):
        video_input = gr.Video(label="Input Cover Video", value = example_cover_video)
        video_input2 = gr.Video(label="Input Secret Video", value = example_secret_video)
        video_output = gr.Video(label="Output Cover Video")
        video_output2 = gr.Video(label="Output Secret Video")
        plot_output = gr.Plot(label="SSIM over Frames for Cover")
        plot_output2 = gr.Plot(label="SSIM over Frames for Secret")
        btn_video = gr.Button("Process Video")

        def process_video_and_plot(video_path, video_path2):
            video_path, video_path2, ssim_scores, ssim_scores2 = steganography_video(video_path, video_path2)
            plot = plot_metrics(ssim_scores)
            plot2 = plot_metrics(ssim_scores2)
            plot.show()
            return video_path, video_path2, plot, plot2

        btn_video.click(
            fn=process_video_and_plot,
            inputs=[video_input, video_input2],
            outputs=[video_output, video_output2, plot_output, plot_output2]
        )

In [ ]:
demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://30516838341b6a8c15.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Saved frame 0 to /content/drive/MyDrive/Machine Learning Techniques/Proyecto/Frames1/frame0.jpg
Saved frame 1 to /content/drive/MyDrive/Machine Learning Techniques/Proyecto/Frames1/frame1.jpg
Saved frame 2 to /content/drive/MyDrive/Machine Learning Techniques/Proyecto/Frames1/frame2.jpg
Saved frame 3 to /content/drive/MyDrive/Machine Learning Techniques/Proyecto/Frames1/frame3.jpg
Saved frame 4 to /content/drive/MyDrive/Machine Learning Techniques/Proyecto/Frames1/frame4.jpg
Saved frame 5 to /content/drive/MyDrive/Machine Learning Techniques/Proyecto/Frames1/frame5.jpg
Saved frame 6 to /content/drive/MyDrive/Machine Learning Techniques/Proyecto/Frames1/frame6.jpg
Saved frame 7 to /content/drive/MyDrive/Machine Learning Techniques/Proyecto/Frames1/frame7.jpg
Saved frame 8 to /content/drive/MyDrive/Machine Learning Techniques/Proyecto/Frames1/frame8.jpg
Saved frame 9 to /content/drive/MyDrive/Machine Learning Techniques/Proyecto/Frames1/frame9.jpg
Saved frame 10 to /content/drive/MyDrive

<ipython-input-9-727f1195f038>:8: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((124, 124), Image.ANTIALIAS)


1/1 [==============================] - 0s 410ms/step
(124, 124, 3)
(124, 124, 3)
(124, 124, 3)
(124, 124, 3)
1/1 [==============================] - 0s 446ms/step
(124, 124, 3)
(124, 124, 3)
(124, 124, 3)
(124, 124, 3)
1/1 [==============================] - 0s 426ms/step
(124, 124, 3)
(124, 124, 3)
(124, 124, 3)
(124, 124, 3)
1/1 [==============================] - 0s 404ms/step
(124, 124, 3)
(124, 124, 3)
(124, 124, 3)
(124, 124, 3)
1/1 [==============================] - 0s 439ms/step
(124, 124, 3)
(124, 124, 3)
(124, 124, 3)
(124, 124, 3)
1/1 [==============================] - 0s 420ms/step
(124, 124, 3)
(124, 124, 3)
(124, 124, 3)
(124, 124, 3)
1/1 [==============================] - 0s 442ms/step
(124, 124, 3)
(124, 124, 3)
(124, 124, 3)
(124, 124, 3)
1/1 [==============================] - 0s 415ms/step
(124, 124, 3)
(124, 124, 3)
(124, 124, 3)
(124, 124, 3)
1/1 [==============================] - 0s 426ms/step
(124, 124, 3)
(124, 124, 3)
(124, 124, 3)
(124, 124, 3)
1/1 [==============

/usr/local/lib/python3.10/dist-packages/gradio/components/video.py:327: UserWarning: Video does not have browser-compatible container or codec. Converting to mp4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/components/video.py:327: UserWarning: Video does not have browser-compatible container or codec. Converting to mp4.
  warnings.warn(


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://30516838341b6a8c15.gradio.live


In [ ]:
demo.launch(debug=True)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://880c03baa98e952de0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


KeyboardInterrupt: 